In [5]:
import datetime
import json
import os
import pickle
from dateutil import parser

import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [12]:
# 리뷰 전처리 함수
def preprocess_reviews(path='yanolja-review\\res\\review_list_리치몬드 호텔 텐진 니시-도리.json'):
    with open(path, 'r', encoding='utf-8') as f:
        reviews = json.load(f)
    return reviews

In [7]:
# 리뷰 요약 함수
def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo-0125', max_reviews=5):
    limited_reviews = reviews[:max_reviews]  # 첫 5개의 리뷰만 선택
    combined_reviews = "\n\n".join([f"리뷰: {review['review']} (별점: {review['stars']}/5)" for review in limited_reviews])
    prompt = prompt + '\n\n' + combined_reviews

    # OpenAI API를 사용해 요약 요청
    completion = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt}],
        temperature=temperature
    )
    
    # 결과 반환
    return completion.choices[0].message.content

In [11]:
# Gradio에서 실행되는 함수
def fn():
    # 리뷰 파일 경로
    path = 'yanolja-review\\res\\review_list_리치몬드 호텔 텐진 니시-도리.json'

    # 리뷰 전처리
    reviews = preprocess_reviews(path)

    # 기본 프롬프트
    PROMPT_BASELINE = "호텔 리뷰를 요약해 주세요."

    # 높은 평점과 낮은 평점으로 나누어 요약
    high_reviews = [r for r in reviews if r['stars'] >= 4]  # 별점 4 이상
    low_reviews = [r for r in reviews if r['stars'] <= 2]  # 별점 2 이하

    # 요약 수행
    high_summary = summarize(high_reviews, PROMPT_BASELINE) if high_reviews else "고평점 리뷰가 없습니다."
    low_summary = summarize(low_reviews, PROMPT_BASELINE) if low_reviews else "저평점 리뷰가 없습니다."

    return high_summary, low_summary

In [9]:
# Gradio 인터페이스
def run_demo():
    demo = gr.Interface(
        fn=fn,
        inputs=[], 
        outputs=[gr.Textbox(label='높은 평점 요약'), gr.Textbox(label='낮은 평점 요약')]
    )
    demo.launch(share=True)

In [13]:
if __name__ == '__main__':
    run_demo()

Running on local URL:  http://127.0.0.1:7861
IMPORTANT: You are using gradio version 4.32.0, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://8d55d003eb701edf63.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [14]:
# 1-shot 예시 프롬프트 작성 및 실행
reviews_1shot = preprocess_reviews(path="yanolja-review\\res\\review_list_리치몬드 호텔 텐진 니시-도리.json")
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

아래 숙소 리뷰들에 대해 요약해주세요:"""

summary_1shot = summarize(reviews_1shot, prompt_1shot, temperature=0.0, model='gpt-4o-2024-05-13')

# 2-shot 프롬프트 작성
prompt_2shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot[:5]}  # 리뷰의 개수를 줄였습니다.
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

reviews = preprocess_reviews(path="yanolja-review\\res\\review_list_리치몬드 호텔 텐진 니시-도리.json")
summary_2shot = summarize(reviews, prompt_2shot, temperature=1.0, model='gpt-4o-2024-05-13')

In [15]:
# Gradio 인터페이스 구성
def display_summaries():
    return summary_1shot, summary_2shot

In [16]:
# Gradio 인터페이스 설정
demo = gr.Interface(
    fn=display_summaries,
    inputs=[],
    outputs=[gr.Textbox(label='1-shot 요약 결과'), gr.Textbox(label='2-shot 요약 결과')]
)

# Gradio 실행
if __name__ == '__main__':
    demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7862
IMPORTANT: You are using gradio version 4.32.0, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://22ec8e1ffd1d355d80.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
